In [1]:
# ! pip install --upgrade --quiet langchain langchain-community langchainhub langchain-openai langchain-text-splitters faiss-cpu 'arize-phoenix[evals]'

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [3]:
import phoenix as px

session = px.launch_app()

/Users/nicktroast/Workspace/ai_agents_cookbooks/.venv/lib/python3.12/site-packages/hdbscan/plots.py:448: SyntaxWarning: invalid escape sequence '\l'
  axis.set_ylabel('$\lambda$ value')
/Users/nicktroast/Workspace/ai_agents_cookbooks/.venv/lib/python3.12/site-packages/hdbscan/robust_single_linkage_.py:154: SyntaxWarning: invalid escape sequence '\{'
  """Perform robust single linkage clustering from a vector array


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [4]:
from phoenix.trace.langchain import LangChainInstrumentor

LangChainInstrumentor().instrument()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
loader = DirectoryLoader("../city_data")
docs = loader.load_and_split(text_splitter=text_splitter)

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mxbai-embed-large")
db = FAISS.from_documents(docs, embeddings)

In [12]:
retriever = db.as_retriever()

In [23]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "city_search",
    "Search for information about cities. For any questions about cities, you must use this tool.",
)
tools = [tool]

In [24]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="llama3", temperature=0, verbose=True, base_url="http://localhost:11434/v1")

In [26]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [28]:
result = agent_executor.invoke(
    {"input": "Which city has the best food?"}
)

In [29]:
result["output"]

'What a delicious question!\n\nThe answer, of course, is subjective and depends on personal taste preferences. However, I can give you some insights based on popular opinions and culinary reputations.\n\nHere are some cities known for their exceptional food scenes:\n\n1. **New Orleans, Louisiana**: Known as the birthplace of jazz, NOLA is also famous for its Creole and Cajun cuisine. Beignets at Cafe du Monde, gumbo, jambalaya, po\' boys, and beignet-filled breakfasts are just a few reasons to visit.\n2. **Tokyo, Japan**: This city is a food lover\'s paradise! From sushi to ramen, tempura to yakitori, Tokyo offers an incredible variety of flavors and textures. Don\'t miss the Tsukiji Fish Market for fresh sushi and seafood.\n3. **Bangkok, Thailand**: Street food heaven! Bangkok\'s night markets and street stalls serve up spicy tom yum soup, fragrant pad thai, and sweet mango sticky rice. Try the famous boat noodles or indulge in a luxurious dinner at one of the many Michelin-starred re